In [1]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd

In [2]:
#상윤님이 보내주는 코드에 이어서 작성해야 함 불러오는 파일은 당연히 원본파일
df = pd.read_csv('../data_refine/refine_final.csv')

df

,LOT,Injection_Velocity_1,Injection_Velocity_2,Injection_Velocity_3,Spray_Time_1,Spray_Time_2,Humidity,Temperature,Casting_Pressure,Pressure_Increase_Time,TYPE
0,180102-021-0101,0.120,0.220,0.368,0.2,0.3,19.993,15.598,680,0.041,0
1,180102-021-0101,0.106,0.216,0.357,0.2,0.3,18.778,14.997,678,0.043,0
2,180102-021-0101,0.085,0.222,0.371,0.2,0.3,20.330,15.835,680,0.043,0
3,180102-021-0101,0.080,0.208,0.349,0.2,0.3,18.382,14.841,324,0.000,0
4,180102-021-0101,0.106,0.214,0.360,0.2,0.3,18.961,15.028,677,0.043,0
...,...,...,...,...,...,...,...,...,...,...,...
8626,180606-022-0101,0.192,0.292,0.408,0.2,0.3,40.344,30.042,341,0.000,2
8627,180606-022-0101,0.188,0.294,0.416,0.2,0.3,38.394,31.105,341,0.000,2
8628,180606-022-0101,0.194,0.294,0.411,0.2,0.3,35.061,31.599,341,0.000,2
8629,180606-022-0101,0.185,0.296,0.416,0.2,0.3,31.495,32.699,340,0.000,2


In [6]:
# 학습데이터 셋 분리

X = df.drop(columns = ['LOT', 'TYPE'])
Y = df[['TYPE']]

In [7]:
X

,Injection_Velocity_1,Injection_Velocity_2,Injection_Velocity_3,Spray_Time_1,Spray_Time_2,Humidity,Temperature,Casting_Pressure,Pressure_Increase_Time
0,0.120,0.220,0.368,0.2,0.3,19.993,15.598,680,0.041
1,0.106,0.216,0.357,0.2,0.3,18.778,14.997,678,0.043
2,0.085,0.222,0.371,0.2,0.3,20.330,15.835,680,0.043
3,0.080,0.208,0.349,0.2,0.3,18.382,14.841,324,0.000
4,0.106,0.214,0.360,0.2,0.3,18.961,15.028,677,0.043
...,...,...,...,...,...,...,...,...,...
8626,0.192,0.292,0.408,0.2,0.3,40.344,30.042,341,0.000
8627,0.188,0.294,0.416,0.2,0.3,38.394,31.105,341,0.000
8628,0.194,0.294,0.411,0.2,0.3,35.061,31.599,341,0.000
8629,0.185,0.296,0.416,0.2,0.3,31.495,32.699,340,0.000


In [8]:
Y

,TYPE
0,0
1,0
2,0
3,0
4,0
...,...
8626,2
8627,2
8628,2
8629,2


In [9]:
from sklearn.model_selection import train_test_split

tr_x, te_x, tr_y, te_y = \
    train_test_split(X, Y, test_size=0.3, random_state=1234)

## 1. random state 로 데이터 학습

In [17]:
params = { 'max_depth':3,
           'eta': 0.1,
           'objective':'multi:softmax',
           'eval_metric':'error : Binary classification error rate (0.5 threshold)'}
num_rounds = 400

# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다.
wlist = [(tr_x, tr_y),(te_x,te_y) ]

In [18]:
xgb_model = xgb.XGBClassifier(objective = 'multi:softmax',
                              num_class = 4,
                              missing = 1,
                              early_stopping_rounds = 10,
                              eval_metric = 'Binary classification error rate (0.5 threshold)'
                              )
xgb_model.fit(tr_x, tr_y, verbose = 1, eval_set = wlist)

# xgboost는 결과가 아닌, 결과를 추정하는 확률로 값이 반환한다.
# pred_probs = xgb_model.predict()
# pred_probs
# preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]



XGBoostError: [22:07:51] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/metric/metric.cc:49: Unknown metric function Binary classification error rate (0.5 threshold)
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000172c68705 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x0000000172d7bd4b xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, xgboost::GenericParameter const*) + 139
  [bt] (2) 3   libxgboost.dylib                    0x0000000172d47c2d xgboost::LearnerConfiguration::ConfigureMetrics(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, 

### https://www.kaggle.com/code/emmanuelfwerr/xgboost-multi-class-classification%C3%A5

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

accuracy = accuracy_score()